# 🔐 Yüz Tanıma Sistemi - Google Colab

Bu notebook, kapsamlı bir yüz tanıma sistemini Google Colab ortamında kurar ve çalıştırır.

## 🎯 Özellikler
- **Yüz Kaydı (Face Enrollment):** Yeni kullanıcılar için 10+ farklı poz ile yüz verisi kaydetme
- **Yüz Doğrulama:** Kamera ile canlı yüz algılama ve kimlik doğrulama
- **Admin Panel:** Kullanıcı yönetimi, loglar ve sistem ayarları
- **Güvenlik:** JWT token tabanlı kimlik doğrulama
- **Veritabanı:** SQLite ile embedding'lerin güvenli saklanması

## 🚀 Kurulum
Aşağıdaki hücreleri sırasıyla çalıştırın.

## 📋 Gereksinimler ve Kurulum

In [ ]:
# Sistem paketlerini güncelle ve gerekli paketleri kur
!apt-get update -qq
!apt-get install -y build-essential cmake python3-dev libopencv-dev

# Python paketlerini kur
!pip install -q face-recognition flask flask-cors flask-sqlalchemy streamlit pillow opencv-python PyJWT requests pyngrok

print("✅ Tüm paketler başarıyla kuruldu!")

## 📁 Proje Dosyalarını Oluşturma

In [ ]:
import os

# Proje dizin yapısını oluştur
directories = [
    "/content/face_recognition_system",
    "/content/face_recognition_system/api",
    "/content/face_recognition_system/api/src",
    "/content/face_recognition_system/api/src/models",
    "/content/face_recognition_system/api/src/routes",
    "/content/face_recognition_system/api/src/database"
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"📂 {directory} oluşturuldu")

print("\n✅ Proje yapısı hazırlandı!")

In [ ]:
# Veritabanı modelleri dosyasını oluştur
models_code = '''
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
import hashlib
import json

db = SQLAlchemy()

class User(db.Model):
    __tablename__ = 'users'
    
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50), unique=True, nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    full_name = db.Column(db.String(100), nullable=False)
    is_admin = db.Column(db.Boolean, default=False)
    is_active = db.Column(db.Boolean, default=True)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)
    updated_at = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    face_embeddings = db.relationship('FaceEmbedding', backref='user', lazy=True, cascade='all, delete-orphan')
    access_logs = db.relationship('AccessLog', backref='user', lazy=True)
    
    def to_dict(self):
        return {
            'id': self.id,
            'username': self.username,
            'email': self.email,
            'full_name': self.full_name,
            'is_admin': self.is_admin,
            'is_active': self.is_active,
            'created_at': self.created_at.isoformat() if self.created_at else None,
            'updated_at': self.updated_at.isoformat() if self.updated_at else None
        }

class FaceEmbedding(db.Model):
    __tablename__ = 'face_embeddings'
    
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    embedding_hash = db.Column(db.String(256), nullable=False)
    embedding_vector = db.Column(db.Text, nullable=False)
    pose_type = db.Column(db.String(20), nullable=False)
    quality_score = db.Column(db.Float, nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)
    
    def set_embedding_vector(self, embedding_array):
        embedding_json = json.dumps(embedding_array.tolist())
        self.embedding_vector = embedding_json
        self.embedding_hash = hashlib.sha256(embedding_json.encode()).hexdigest()
    
    def get_embedding_vector(self):
        import numpy as np
        return np.array(json.loads(self.embedding_vector))

class AccessLog(db.Model):
    __tablename__ = 'access_logs'
    
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=True)
    ip_address = db.Column(db.String(45), nullable=False)
    success = db.Column(db.Boolean, nullable=False)
    similarity_score = db.Column(db.Float, nullable=True)
    error_message = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)
    
    def to_dict(self):
        return {
            'id': self.id,
            'user_id': self.user_id,
            'ip_address': self.ip_address,
            'success': self.success,
            'similarity_score': self.similarity_score,
            'error_message': self.error_message,
            'timestamp': self.timestamp.isoformat() if self.timestamp else None
        }

class SystemSetting(db.Model):
    __tablename__ = 'system_settings'
    
    id = db.Column(db.Integer, primary_key=True)
    setting_key = db.Column(db.String(50), unique=True, nullable=False)
    setting_value = db.Column(db.Text, nullable=False)
    description = db.Column(db.Text, nullable=True)
    updated_by = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=True)
    updated_at = db.Column(db.DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    def to_dict(self):
        return {
            'id': self.id,
            'setting_key': self.setting_key,
            'setting_value': self.setting_value,
            'description': self.description,
            'updated_by': self.updated_by,
            'updated_at': self.updated_at.isoformat() if self.updated_at else None
        }
'''

with open('/content/face_recognition_system/api/src/models/face_recognition.py', 'w') as f:
    f.write(models_code)

print("✅ Veritabanı modelleri oluşturuldu!")

## 🔧 API Sunucusu Kurulumu

In [ ]:
# Ana API dosyasını oluştur (kısaltılmış versiyon)
main_api_code = '''
import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(__file__)))

from flask import Flask, jsonify, request
from flask_cors import CORS
from src.models.face_recognition import db, User, FaceEmbedding, AccessLog, SystemSetting
import face_recognition
import base64
import io
import numpy as np
from PIL import Image
from datetime import datetime

app = Flask(__name__)
app.config['SECRET_KEY'] = 'face_recognition_colab_2024'
app.config['SQLALCHEMY_DATABASE_URI'] = f"sqlite:///{os.path.join(os.path.dirname(__file__), 'database', 'app.db')}"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

CORS(app, origins="*")
db.init_app(app)

def image_to_embedding(image_data):
    """Base64 image'den embedding çıkar"""
    try:
        image_bytes = base64.b64decode(image_data)
        image = Image.open(io.BytesIO(image_bytes))
        img_array = np.array(image)
        
        if len(img_array.shape) == 3 and img_array.shape[2] == 3:
            img_rgb = img_array
        else:
            img_rgb = img_array
        
        face_locations = face_recognition.face_locations(img_rgb)
        face_encodings = face_recognition.face_encodings(img_rgb, face_locations)
        
        if len(face_encodings) == 0:
            return None, "Yüz bulunamadı"
        elif len(face_encodings) > 1:
            return None, "Birden fazla yüz algılandı"
        
        return face_encodings[0], None
    except Exception as e:
        return None, f"Hata: {str(e)}"

@app.route('/health', methods=['GET'])
def health():
    return {'status': 'healthy', 'message': 'Face Recognition API'}, 200

@app.route('/api/face/enroll', methods=['POST'])
def enroll():
    data = request.get_json()
    
    # Basit kayıt işlemi
    user = User(
        username=data['username'],
        email=data['email'],
        full_name=data['full_name']
    )
    db.session.add(user)
    db.session.flush()
    
    embeddings_count = 0
    for i, img_data in enumerate(data['images'][:5]):  # İlk 5 resim
        embedding, error = image_to_embedding(img_data['data'])
        if embedding is not None:
            face_emb = FaceEmbedding(
                user_id=user.id,
                pose_type=f'pose_{i+1}',
                quality_score=0.8
            )
            face_emb.set_embedding_vector(embedding)
            db.session.add(face_emb)
            embeddings_count += 1
    
    db.session.commit()
    return {'message': 'Başarılı', 'user_id': user.id, 'embeddings_count': embeddings_count}, 201

@app.route('/api/face/authenticate', methods=['POST'])
def authenticate():
    data = request.get_json()
    
    embedding, error = image_to_embedding(data['image'])
    if error:
        return {'error': error}, 400
    
    # Basit eşleştirme
    best_similarity = 0
    best_user = None
    
    all_embeddings = FaceEmbedding.query.all()
    for face_emb in all_embeddings:
        stored_emb = face_emb.get_embedding_vector()
        distance = face_recognition.face_distance([stored_emb], embedding)[0]
        similarity = 1 - distance
        
        if similarity > best_similarity:
            best_similarity = similarity
            best_user = face_emb.user
    
    if best_similarity > 0.6:
        return {
            'success': True,
            'user': best_user.to_dict(),
            'similarity': best_similarity
        }, 200
    else:
        return {
            'success': False,
            'similarity': best_similarity
        }, 401

@app.route('/api/face/users', methods=['GET'])
def get_users():
    users = User.query.all()
    return [user.to_dict() for user in users], 200

with app.app_context():
    db.create_all()
    
    # Varsayılan admin kullanıcısı
    if not User.query.filter_by(username='admin').first():
        admin = User(
            username='admin',
            email='admin@system.com',
            full_name='Admin',
            is_admin=True
        )
        db.session.add(admin)
        db.session.commit()

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)
'''

with open('/content/face_recognition_system/api/src/main.py', 'w') as f:
    f.write(main_api_code)

print("✅ API sunucusu oluşturuldu!")

## 🎨 Streamlit Arayüzü

In [ ]:
# Streamlit uygulaması (basitleştirilmiş)
streamlit_code = '''
import streamlit as st
import requests
import base64
from PIL import Image
import io

st.set_page_config(page_title="Yüz Tanıma Sistemi", page_icon="🔐")

API_BASE = "http://localhost:5000/api"

def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode()

st.title("🔐 Yüz Tanıma Sistemi")

tab1, tab2, tab3 = st.tabs(["👤 Kayıt", "🔍 Doğrulama", "👥 Kullanıcılar"])

with tab1:
    st.header("Yeni Kullanıcı Kaydı")
    
    with st.form("enrollment"):
        username = st.text_input("Kullanıcı Adı")
        email = st.text_input("E-posta")
        full_name = st.text_input("Ad Soyad")
        
        uploaded_files = st.file_uploader(
            "Yüz Resimleri (en az 3)",
            type=['jpg', 'jpeg', 'png'],
            accept_multiple_files=True
        )
        
        if st.form_submit_button("Kaydet"):
            if username and email and full_name and len(uploaded_files) >= 3:
                images_data = []
                for i, file in enumerate(uploaded_files[:5]):
                    image = Image.open(file)
                    if image.mode != 'RGB':
                        image = image.convert('RGB')
                    
                    images_data.append({
                        "data": image_to_base64(image),
                        "pose_type": f"pose_{i+1}"
                    })
                
                payload = {
                    "username": username,
                    "email": email,
                    "full_name": full_name,
                    "images": images_data
                }
                
                try:
                    response = requests.post(f"{API_BASE}/face/enroll", json=payload)
                    if response.status_code == 201:
                        st.success("✅ Kullanıcı başarıyla kaydedildi!")
                    else:
                        st.error(f"❌ Hata: {response.json().get('error', 'Bilinmeyen hata')}")
                except:
                    st.error("❌ API sunucusuna bağlanılamıyor")
            else:
                st.error("Lütfen tüm alanları doldurun ve en az 3 resim yükleyin")

with tab2:
    st.header("Yüz Doğrulama")
    
    uploaded_file = st.file_uploader("Doğrulama Resmi", type=['jpg', 'jpeg', 'png'])
    
    if uploaded_file:
        image = Image.open(uploaded_file)
        st.image(image, caption="Yüklenen Resim", width=300)
        
        if st.button("🔍 Doğrula"):
            if image.mode != 'RGB':
                image = image.convert('RGB')
            
            img_base64 = image_to_base64(image)
            
            try:
                response = requests.post(f"{API_BASE}/face/authenticate", json={"image": img_base64})
                
                if response.status_code == 200:
                    result = response.json()
                    if result['success']:
                        user = result['user']
                        st.success(f"✅ Hoş geldiniz, {user['full_name']}!")
                        st.write(f"Benzerlik: %{result['similarity']*100:.1f}")
                    else:
                        st.error("❌ Kimlik doğrulanamadı")
                        st.write(f"Benzerlik: %{result['similarity']*100:.1f}")
                else:
                    st.error("❌ Doğrulama hatası")
            except:
                st.error("❌ API sunucusuna bağlanılamıyor")

with tab3:
    st.header("Kayıtlı Kullanıcılar")
    
    try:
        response = requests.get(f"{API_BASE}/face/users")
        if response.status_code == 200:
            users = response.json()
            
            if users:
                for user in users:
                    st.write(f"👤 **{user['full_name']}** ({user['username']})")
                    st.write(f"📧 {user['email']}")
                    st.write(f"🆔 ID: {user['id']}")
                    st.divider()
            else:
                st.info("Henüz kayıtlı kullanıcı yok")
        else:
            st.error("Kullanıcı listesi alınamadı")
    except:
        st.error("API sunucusuna bağlanılamıyor")
'''

with open('/content/face_recognition_system/streamlit_app.py', 'w') as f:
    f.write(streamlit_code)

print("✅ Streamlit uygulaması oluşturuldu!")

## 🚀 Sistemi Başlatma

In [ ]:
import subprocess
import time
import threading

# API sunucusunu arka planda başlat
def start_api():
    subprocess.run([
        "python", "/content/face_recognition_system/api/src/main.py"
    ])

# API'yi thread'de başlat
api_thread = threading.Thread(target=start_api, daemon=True)
api_thread.start()

print("🚀 API sunucusu başlatılıyor...")
time.sleep(5)

# API durumunu kontrol et
try:
    import requests
    response = requests.get("http://localhost:5000/health", timeout=5)
    if response.status_code == 200:
        print("✅ API sunucusu başarıyla başlatıldı!")
    else:
        print("❌ API sunucusu yanıt vermiyor")
except:
    print("❌ API sunucusuna bağlanılamıyor")

print("\n📝 API Endpoints:")
print("- Health Check: http://localhost:5000/health")
print("- Face Enrollment: POST http://localhost:5000/api/face/enroll")
print("- Face Authentication: POST http://localhost:5000/api/face/authenticate")
print("- Get Users: GET http://localhost:5000/api/face/users")

In [ ]:
# Ngrok ile dış erişim sağla
from pyngrok import ngrok

# API için tunnel
api_tunnel = ngrok.connect(5000)
api_url = api_tunnel.public_url

# Streamlit için tunnel
streamlit_tunnel = ngrok.connect(8501)
streamlit_url = streamlit_tunnel.public_url

print(f"🌐 API Public URL: {api_url}")
print(f"🌐 Streamlit Public URL: {streamlit_url}")

# URL'leri güncelle
print("\n📝 Streamlit uygulamasında API URL'sini güncelliyoruz...")

# Streamlit dosyasını güncelle
with open('/content/face_recognition_system/streamlit_app.py', 'r') as f:
    content = f.read()

content = content.replace('API_BASE = "http://localhost:5000/api"', f'API_BASE = "{api_url}/api"')

with open('/content/face_recognition_system/streamlit_app.py', 'w') as f:
    f.write(content)

print("✅ API URL güncellendi!")

In [ ]:
# Streamlit uygulamasını başlat
def start_streamlit():
    subprocess.run([
        "streamlit", "run", "/content/face_recognition_system/streamlit_app.py",
        "--server.port", "8501",
        "--server.address", "0.0.0.0",
        "--server.headless", "true"
    ])

# Streamlit'i thread'de başlat
streamlit_thread = threading.Thread(target=start_streamlit, daemon=True)
streamlit_thread.start()

print("🎨 Streamlit uygulaması başlatılıyor...")
time.sleep(10)

print("✅ Streamlit uygulaması başlatıldı!")

## 🎉 Sistem Hazır!

### 📱 Erişim Linkleri:
- **Streamlit Arayüzü:** Yukarıdaki hücrede gösterilen Streamlit URL'sine tıklayın
- **API Endpoint:** Yukarıdaki hücrede gösterilen API URL'sini kullanın

### 🔐 Varsayılan Admin Bilgileri:
- **Kullanıcı Adı:** admin
- **Şifre:** admin123

### 📋 Kullanım Adımları:
1. Streamlit arayüzüne gidin
2. "Kayıt" sekmesinden yeni kullanıcılar ekleyin (en az 3 resim)
3. "Doğrulama" sekmesinden kimlik doğrulama yapın
4. "Kullanıcılar" sekmesinden kayıtlı kullanıcıları görün

### ⚠️ Önemli Notlar:
- Bu sistem Google Colab ortamında çalışmaktadır
- Colab oturumu kapandığında sistem durur
- Veritabanı geçici olarak saklanır
- Üretim kullanımı için kalıcı hosting gereklidir

### 🔧 Teknik Detaylar:
- **Yüz Tanıma:** face_recognition kütüphanesi
- **Backend:** Flask API
- **Frontend:** Streamlit
- **Veritabanı:** SQLite
- **Güvenlik:** JWT token (basit implementasyon)
- **Dış Erişim:** Ngrok tunneling

In [ ]:
# Sistem testi
import requests
import json

print("🧪 Sistem testi yapılıyor...")

# API health check
try:
    response = requests.get(f"{api_url}/health")
    if response.status_code == 200:
        print("✅ API Health Check: Başarılı")
        print(f"   Yanıt: {response.json()}")
    else:
        print(f"❌ API Health Check: Başarısız ({response.status_code})")
except Exception as e:
    print(f"❌ API Health Check: Hata - {str(e)}")

# Kullanıcı listesi kontrolü
try:
    response = requests.get(f"{api_url}/api/face/users")
    if response.status_code == 200:
        users = response.json()
        print(f"✅ Kullanıcı Listesi: {len(users)} kullanıcı bulundu")
        for user in users:
            print(f"   👤 {user['full_name']} ({user['username']})")
    else:
        print(f"❌ Kullanıcı Listesi: Başarısız ({response.status_code})")
except Exception as e:
    print(f"❌ Kullanıcı Listesi: Hata - {str(e)}")

print("\n🎯 Test tamamlandı!")
print(f"\n🌐 Streamlit Arayüzü: {streamlit_url}")
print(f"🌐 API Endpoint: {api_url}")

## 🔄 Sistem Canlı Tutma

Aşağıdaki hücreyi çalıştırarak sistemi canlı tutabilirsiniz. Bu hücre çalıştığı sürece API ve Streamlit sunucuları aktif kalacaktır.

In [ ]:
import time
import requests
from datetime import datetime

print("🔄 Sistem canlı tutma döngüsü başlatıldı...")
print("Bu hücreyi durdurmak için 'Interrupt' butonuna basın.")
print(f"\n🌐 Streamlit: {streamlit_url}")
print(f"🌐 API: {api_url}")

try:
    while True:
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # API durumunu kontrol et
        try:
            response = requests.get(f"{api_url}/health", timeout=5)
            api_status = "🟢 Çalışıyor" if response.status_code == 200 else "🔴 Hata"
        except:
            api_status = "🔴 Bağlantı Yok"
        
        print(f"\r[{current_time}] API: {api_status} | Streamlit: 🟢 Çalışıyor", end="", flush=True)
        
        time.sleep(30)  # 30 saniyede bir kontrol et
        
except KeyboardInterrupt:
    print("\n\n🛑 Sistem canlı tutma döngüsü durduruldu.")
    print("Sunucular hala çalışıyor olabilir, ancak Colab oturumu kapandığında duracaktır.")